In [13]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import experimentos

import importlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import pandas as pd
import json
importlib.reload(experimentos)

<module 'experimentos' from 'c:\\dev\\Busca Heuristica\\T2-BH\\T2\\experimentos.py'>

# Carregamento

In [14]:
def numpy_to_python(obj):
    '''Converte np.int e np.float para int e float dos dados do dicionário'''
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    
def python_to_numpy(dicionario):
    '''Converte int e float para np.int e np.float dos dados do dicionário'''
    for chave, subdict in dicionario.items():
        for subchave, valor in subdict.items():
            if isinstance(valor, int):
                subdict[subchave] = np.int64(valor)
            elif isinstance(valor, float):
                subdict[subchave] = np.float64(valor)
                
    dicionario = {np.int64(key): valor for key, valor in dicionario.items()}
    return dicionario

In [15]:
# Carrega os experimentos do SA
num_calculos_obj = [5000, 10000, 25000, 50000]

caminho_v1 = 'bkp_resultados_sa/sa_v1_experimentos.json'
with open(caminho_v1, 'r', encoding='utf-8') as arquivo:
    v1 = python_to_numpy(json.load(arquivo))
    v1_resultados = experimentos.experimento(v1, num_calculos_obj)
    v1_resultados.calcula_estatisticas()
    
caminho_v2 = 'bkp_resultados_sa/sa_v2_experimentos.json'
with open(caminho_v2, 'r', encoding='utf-8') as arquivo:
    v2 = python_to_numpy(json.load(arquivo))
    v2_resultados = experimentos.experimento(v2, num_calculos_obj)
    v2_resultados.calcula_estatisticas()
    
caminho_v3 = 'bkp_resultados_sa/sa_v3_experimentos.json'
with open(caminho_v3, 'r', encoding='utf-8') as arquivo:
    v3 = python_to_numpy(json.load(arquivo))
    v3_resultados = experimentos.experimento(v3, num_calculos_obj)
    v3_resultados.calcula_estatisticas()

# Resultados Simulated Annealing

## Comparações numéricas das vizinhanças e métodos com valor de referência

Valor de referência: R$ 32.170.883

$$Eficiência = \frac{VPL_{Médio}}{VPL_{Referência}}$$

In [ ]:
def dicionarios_para_dataframe(dicionarios, nomes):
    dados = []
    indices = []
    for nome, dicionario in zip(nomes, dicionarios):
        for num_calculos, metodos in dicionario.items():
            for metodo, valores in metodos.items():
                indices.append((nome, metodo, num_calculos))
                dados.append([valores["media"], valores["maximo"], valores["minimo"]])
    # Criar o DataFrame com índice no formato de tuplas
    df = pd.DataFrame(dados, 
                      index=pd.MultiIndex.from_tuples(indices, names=["Vn", "Metodo", "NumCalculos"]),
                      columns=["Media", "Maximo", "Minimo"])
    
    # Calcular a eficiência (Média / 32170883)
    df["Eficiência"] = df["Media"] / 32170883
    
    # Definir a exibição de números para evitar notação científica
    pd.set_option('display.float_format', '{:,.3f}'.format)
    
    return df


# Criar o DataFrame
dfs = [v1_resultados.estatisticas, v2_resultados.estatisticas, v3_resultados.estatisticas]
nomes = ["Aleatória", "Localizada", "Sistemática"]
df = dicionarios_para_dataframe(dfs, nomes)

# Exibir o DataFrame
df
df.groupby(["Vn", "Metodo", "NumCalculos"]).mean()

## Comparação dos métodos MM e PM para cada vizinhança

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(8,16))
plt.rcParams.update({'font.size': 10})

plt.sca(axs[0])
v1_resultados.plota_experimento(medida='media', vizinhanca='Aleatória')
plt.sca(axs[1])
v2_resultados.plota_experimento(medida='media', vizinhanca='Localizada')
plt.sca(axs[2])
v3_resultados.plota_experimento(medida='media', vizinhanca='Sistemática')

plt.show()

## Comparação do VPL médio das diferentes vizinhanças

**Olhar quem é melhor antes de plotar aqui (MM ou PM)!!!**

In [18]:
# Pega melhor método de cada experimento

v1_mm_media = []
for chave, subdict in v1_resultados.estatisticas.items():
    for subchave, valor in subdict.items():
        if subchave == 'mm':
            v1_mm_media.append(valor.get('media'))
            
v2_mm_media = []
for chave, subdict in v2_resultados.estatisticas.items():
    for subchave, valor in subdict.items():
        if subchave == 'mm':
            v2_mm_media.append(valor.get('media'))
            
v3_pm_media = []
for chave, subdict in v3_resultados.estatisticas.items():
    for subchave, valor in subdict.items():
        if subchave == 'pm':
            v3_pm_media.append(valor.get('media'))

In [ ]:
plt.plot(v1_resultados.num_calculos_obj, 
         v1_mm_media, 
         label='Aleatória (MM)', 
         marker='o')
plt.plot(v2_resultados.num_calculos_obj, 
         v2_mm_media, 
         label='Localizada (MM)', 
         marker='o')
plt.plot(v3_resultados.num_calculos_obj, 
         v3_pm_media, 
         label='Sistemaática (PM)', 
         marker='o')

plt.title('VPL médio por número de cálculos de função objetivo')
plt.xlabel('Número de cálculos de função objetivo', fontsize=12)
plt.ylabel('VPL', fontsize=12)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
plt.ticklabel_format(style='plain', axis='x') 
plt.legend()
plt.grid()
plt.show()